In [ ]:
import subprocess
import os



# Go up one directory level from the current path
VIDEO_PATH = "C:\\Users\\es25591\\Workspace\\VideoPredict360\\SampleVideos"
RAW_VIDEO_FILE = VIDEO_PATH + "\\1_Rhinos.mp4"

OUTPUT_PATH = "C:\\Users\\es25591\\Workspace\\VideoPredict360\\EncodedVideoPool"  # This directory is the "Encoded Video Pool" 
GOP_DURATION_SEC = 2  # Videos are broken into segments/GOPs [cite: 167]

# The articles mention L quality layers and M tiles[cite: 164].
# One paper uses M=12 tiles and L=2 quality layers as an example[cite: 400].
# It also specifies bitrates for a base and enhancement layer[cite: 401].
QUALITY_LAYERS = {
    "base_layer": "2M",       # [cite: 401]
    "enhancement_layer": "12M"  # [cite: 401]
}
TILE_GRID = "4x3"  # A 4x3 grid equals 12 tiles [cite: 400]

# --- 2. Create the "Pool" Directory ---
print(f"Creating the 'Encoded Video Pool' at: {OUTPUT_PATH}")
os.makedirs(OUTPUT_PATH, exist_ok=True)

Creating the 'Encoded Video Pool' at: C:\Users\es25591\Workspace\VideoPredict360\EncodedVideoPool


In [ ]:
ffprobe_command = [
    "C:\\Users\\es25591\\Workspace\\ffmpeg-8.0\\bin\\ffprobe.exe",
    "-v", "quiet",
    "-print_format", "json",
    "-show_format",
    "-show_streams",
    RAW_VIDEO_FILE
]

print(" ".join(ffprobe_command))

try:
    result = subprocess.run(ffprobe_command, capture_output=True, text=True)
    video_info = result.stdout
    print("Video File Details:")
    print("-" * 50)
    print(video_info)
except subprocess.CalledProcessError as e:
    print(f"Error getting video details: {e}")
except FileNotFoundError:
    print("Error: ffprobe not found")

C:\Users\es25591\Workspace\ffmpeg-8.0\bin\ffprobe.exe -v quiet -print_format json -show_format -show_streams C:\Users\es25591\Workspace\VideoPredict360\SampleVideos\1_Rhinos.mp4
Video File Details:
--------------------------------------------------
{
    "streams": [
        {
            "index": 0,
            "codec_name": "h264",
            "codec_long_name": "H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10",
            "profile": "High",
            "codec_type": "video",
            "codec_tag_string": "avc1",
            "codec_tag": "0x31637661",
            "width": 1920,
            "height": 1080,
            "coded_width": 1920,
            "coded_height": 1080,
            "has_b_frames": 2,
            "sample_aspect_ratio": "135:128",
            "display_aspect_ratio": "15:8",
            "pix_fmt": "yuv420p",
            "level": 40,
            "chroma_location": "left",
            "field_order": "progressive",
            "refs": 1,
            "is_avc": "true",
       

In [ ]:
TILE_GRID = "4x3" 

print("Starting transcoding process (Tiling, Layering, Segmenting)...")

ffmpeg_command = [
    "C:\\Users\\es25591\\Workspace\\ffmpeg-8.0\\bin\\ffmpeg",
    "-i", RAW_VIDEO_FILE,
    "-c:v", "libx265",
    "-x265-params", f"tiles={TILE_GRID}:tile-for-enabling=1",
    "-keyint_min", str(GOP_DURATION_SEC * 30),
    "-g", str(GOP_DURATION_SEC * 30),
    "-sc_threshold", "0",
    "-map", "0:v:0", "-b:v:0", QUALITY_LAYERS["base_layer"],
    "-map", "0:v:0", "-b:v:1", QUALITY_LAYERS["enhancement_layer"],
    "-f", "dash",
    "-seg_duration", str(GOP_DURATION_SEC),
    "-use_template", "1",
    "-use_timeline", "1",
    "-init_seg_name", f"{OUTPUT_PATH}//init_stream$RepresentationID$.m4s",
    "-media_seg_name", f"{OUTPUT_PATH}//chunk_stream$RepresentationID$_$Number%05d$.m4s",
    os.path.join(OUTPUT_PATH, "stream.mpd")
]

# --- 4. Execute the Command via Python Subprocess ---
try:
    # This is the line where Python actually runs the encoder
    subprocess.run(ffmpeg_command, check=True)
    
    print("\n------------------------------------------------")
    print(f"Successfully encoded! The 'Encoded Video Pool' is ready.")
    print(f"Directory: {OUTPUT_PATH}")
    print("Pool Contents:")
    
    # List the contents of the "pool"
    for f in os.listdir(OUTPUT_PATH):
        print(f"  - {f}")

except subprocess.CalledProcessError as e:
    print(f"\nAn error occurred during encoding: {e}")
except FileNotFoundError:
    print("\nError: FFmpeg not found.")
    print("Please ensure FFmpeg is installed and accessible in your system's PATH.")

Starting transcoding process (Tiling, Layering, Segmenting)...
